In [2]:
import pandas as pd
import nltk
import re
import numpy as np
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier


from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier

C:\Users\Glenn\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [64]:
headers = ['ID', 'TEXT', 'ASPECT_TERM', 'LOCATION']
df = pd.read_csv('Data-1_test.csv', header=None,skiprows=1)
df.columns = headers

In [65]:
df.head()

,ID,TEXT,ASPECT_TERM,LOCATION
0,494:1_0,Buy the separate RAM memory and you will have ...,RAM memory,17--27
1,311:18_1,But with this laptop[comma] the bass is very w...,sound,52--57
2,256:1_0,This hardware seems to be better than the iMac...,hardware,5--13
3,748:1_0,It is very easy to navigate even for a novice.,navigate,19--27
4,128:1_0,Amazing Performance for anything I throw at it.,Performance,8--19


In [66]:
print (len(df.index))

638


In [67]:
text = []
for index, row in df.iterrows():
    #     converting to lowercase, replacing [comma], replacing underscores and removing punctuations
    text.append(str(row['TEXT']).replace('[comma]',''))
#     text.append(re.sub(r'[^\w\s]','',str(row['TEXT']).replace('[comma]','').replace('_','')))
print(text)

['Buy the separate RAM memory and you will have a rocket!', 'But with this laptop the bass is very weak and the sound comes out sounding tinny.', "This hardware seems to be better than the iMac in that it isn't $1400 and smaller.", 'It is very easy to navigate even for a novice.', 'Amazing Performance for anything I throw at it.', 'also...- excellent operating system- size and weight for optimal mobility- excellent durability of the battery- the functions provided by the trackpad is unmatched by any other brand-', 'Chatting with Acer support I was advised the problem was corrupted operating system files.', 'My wife was so excited to open the box but quickly came to see that it did not function as it should.', 'It was important that it was powerful enough to do all of the tasks he needed on the internet word processing graphic design and gaming.', "It's great for streaming video and other entertainment uses.", 'The sound was crappy even when you turn up the volume still the same results

In [68]:
new_text_list = []
for each_text in text:
    new_text_list.append(each_text.split())
print(new_text_list)

[['Buy', 'the', 'separate', 'RAM', 'memory', 'and', 'you', 'will', 'have', 'a', 'rocket!'], ['But', 'with', 'this', 'laptop', 'the', 'bass', 'is', 'very', 'weak', 'and', 'the', 'sound', 'comes', 'out', 'sounding', 'tinny.'], ['This', 'hardware', 'seems', 'to', 'be', 'better', 'than', 'the', 'iMac', 'in', 'that', 'it', "isn't", '$1400', 'and', 'smaller.'], ['It', 'is', 'very', 'easy', 'to', 'navigate', 'even', 'for', 'a', 'novice.'], ['Amazing', 'Performance', 'for', 'anything', 'I', 'throw', 'at', 'it.'], ['also...-', 'excellent', 'operating', 'system-', 'size', 'and', 'weight', 'for', 'optimal', 'mobility-', 'excellent', 'durability', 'of', 'the', 'battery-', 'the', 'functions', 'provided', 'by', 'the', 'trackpad', 'is', 'unmatched', 'by', 'any', 'other', 'brand-'], ['Chatting', 'with', 'Acer', 'support', 'I', 'was', 'advised', 'the', 'problem', 'was', 'corrupted', 'operating', 'system', 'files.'], ['My', 'wife', 'was', 'so', 'excited', 'to', 'open', 'the', 'box', 'but', 'quickly', 'c

In [69]:
# for i in range(len(text)):
#     text[i] = nltk.word_tokenize(text[i])
#     for token in text[i]:
#         if token in stopwords.words('english?)
#             text[i].remove(token)


In [70]:
aspect_terms = []
for index, row in df.iterrows():
    aspect_terms.append(str(row['ASPECT_TERM']))
print(aspect_terms)

['RAM memory', 'sound', 'hardware', 'navigate', 'Performance', 'weight', 'operating system files', 'function', 'gaming', 'streaming video', 'volume', 'durability of the battery', 'playing steam games', 'form factor', 'operating system', 'HD', 'transport', 'keyboard', 'Windows 7', 'Hardware performance', 'watching videos', 'install', 'UX', 'Windows updates', 'quality', 'components', 'aluminum', 'Photoshop', 'iPhoto', 'virtual home schooling', 'built in screen size', 'Windows 8', 'body', 'word processing', 'size', 'time response', 'downloading apps', 'click pads', 'use', 'set up', 'sound', 'disc drive', 'external microphones', 'features', 'battery life', 'unibody construction', 'works', 'Audio', 'MagSafe accessories', 'battery life', 'CD slot', 'cover', 'apps', 'log on', 'design', 'HDMI receptacle', 'functionality', 'ports', 'graphic design', 'presentation', 'OS', 'practicality', '21" LED screen', 'MS Office 2011 for Mac', 'Quad-Core 2.5 GHz CPU', 'Quicken', 'finish', 'mouse', 'keyboard'

In [71]:
# text_context_list = []
# i = 0
# for aspect_term in aspect_terms:
#     print(aspect_term)
#     print(new_text_list[i])
#     if aspect_term in new_text_list[i]:
#         index = new_text_list[i].index(aspect_term)
#         print(index)
#         text_context_list.append(new_text_list[i][index-3:index+4])
#     i+=1
# print(text_context_list)
        

In [72]:
i = 0
reduced_text_list = []
for aspect_term in aspect_terms:
    if aspect_term in text[i]:
        index = text[i].index(aspect_term)
        print(index)
#         new_text_list.append(text[i][index-4:index+5])
    else:
        print(aspect_term," ", text[i], i+1)
    i+=1

17
51
5
19
8
46
67
79
129
15
47
85
124
11
19
19
23
82
14
29
98
45
49
39
40
29
60
74
64
42
74
52
11
94
12
49
59
30
23
40
4
35
98
48
45
10
120
38
31
66
23
6
28
27
73
12
80
8
110
32
83
11
20
0
90
66
41
137
12
75
25
0
42
35
82
8
43
73
23
39
113
26
7
35
95
6
34
50
0
4
34
5
28
24
36
88
9
19
45
30
0
65
16
76
7
0
22
62
43
15
27
12
6
40
38
31
3
0
5
4
23
71
20
7
119
193
8
40
8
68
30
3
26
50
40
17
65
62
25
58
16
116
62
36
58
27
8
154
85
17
58
22
77
0
42
48
9
29
37
23
86
118
94
4
4
15
8
40
53
42
20
72
13
48
55
31
37
89
97
18
25
50
13
4
0
36
52
72
64
60
19
0
36
0
15
24
0
0
244
79
19
56
23
47
21
0
49
37
48
11
37
44
19
0
3
81
70
6
4
47
12
72
0
84
0
73
17
43
4
14
9
12
44
26
47
50
29
25
27
0
22
26
55
38
19
39
43
76
45
45
0
12
55
9
67
38
80
31
55
56
0
58
76
77
5
72
25
86
0
84
25
44
7
25
12
0
4
46
26
4
3
37
4
72
23
49
55
32
24
4
48
34
0
47
190
18
14
183
38
14
50
0
4
20
0
29
22
4
8
38
56
4
67
47
103
25
10
37
35
83
69
15
21
27
113
16
27
29
38
4
0
10
33
41
53
51
26
3
WLAN card   I re-seated the "WLAN" card 

# Decision Tree Classifier

In [77]:
X = TfidfVectorizer(analyzer=preprocess).fit_transform(text)
y = df['CLASS']
dt_model = DecisionTreeClassifier(max_depth=10).fit(X,y)

In [86]:
pred_c = cross_val_predict(dt_model,X,y,cv=10)
# print(X,pred_c)
# np.mean(y == pred_c)
print(precision_recall_fscore_support(y, pred_c, labels=[-1,0,1]))

# print("\n Classification Report \n ", classification_report(pred_c,y))

(array([0.4735328 , 0.55555556, 0.84452975]), array([0.95034642, 0.06521739, 0.44579534]), array([0.63210445, 0.11673152, 0.58355438]), array([866, 460, 987], dtype=int64))


# Linear SVC

In [79]:
svc_model = LinearSVC().fit(X,y)
pred_svc = cross_val_predict(svc_model,X,y,cv=10)
np.mean(y == pred_svc)

0.7181150021616948

In [80]:
print(precision_recall_fscore_support(y, pred_svc, labels=[-1,0,1]))

print("\n Classification Report \n ", classification_report(pred_svc,y))

(array([0.71459227, 0.56382979, 0.77910448]), array([0.76905312, 0.46086957, 0.79331307]), array([0.74082314, 0.50717703, 0.78614458]), array([866, 460, 987], dtype=int64))

 Classification Report 
               precision    recall  f1-score   support

         -1       0.77      0.71      0.74       932
          0       0.46      0.56      0.51       376
          1       0.79      0.78      0.79      1005

avg / total       0.73      0.72      0.72      2313



# Random Forest Classifier

In [81]:
rfc_model = RandomForestClassifier(max_depth=10).fit(X,y)
pred_rfc = cross_val_predict(rfc_model,X,y,cv=10)
np.mean(y == pred_rfc)

0.5754431474275832

In [82]:
print(precision_recall_fscore_support(y, pred_rfc, labels=[-1,0,1]))

print("\n Classification Report \n ", classification_report(pred_rfc,y))

(array([0.61165049, 0.68333333, 0.55417755]), array([0.50923788, 0.08913043, 0.86018237]), array([0.5557656 , 0.15769231, 0.67407701]), array([866, 460, 987], dtype=int64))

 Classification Report 
               precision    recall  f1-score   support

         -1       0.51      0.61      0.56       721
          0       0.09      0.68      0.16        60
          1       0.86      0.55      0.67      1532

avg / total       0.73      0.58      0.62      2313



# Logistic Regression

In [83]:
lgr = LogisticRegression(random_state=0,solver='lbfgs',multi_class='multinomial').fit(X,y)
pred_lgr = cross_val_predict(lgr,X,y,cv=10)
np.mean(y == pred_rfc)

0.5754431474275832

In [84]:
print(precision_recall_fscore_support(y, pred_lgr, labels=[-1,0,1]))

print("\n Classification Report \n ", classification_report(pred_lgr,y))

(array([0.7122449 , 0.65217391, 0.76388889]), array([0.80600462, 0.35869565, 0.83586626]), array([0.75622969, 0.4628331 , 0.79825835]), array([866, 460, 987], dtype=int64))

 Classification Report 
               precision    recall  f1-score   support

         -1       0.81      0.71      0.76       980
          0       0.36      0.65      0.46       253
          1       0.84      0.76      0.80      1080

avg / total       0.77      0.73      0.74      2313

